This file is a simple test of the ODE integrator in SciPy and my ability to write a system of ODEs. The system is a very simplified version of $H_2$-$O_2$ chemistry. The system of reactions to be solved involves 6 reactions (3 reversible, 6 irreversible) among 6 species: $H$, $O$, $OH$, $H_2$, $O_2$, and $H_2O$. The 3 reversible reactions are assumed to be in partial equilibrium. The derivation follows the discussion of Turns, Pg. 133.

In [ ]:
%%latex
The reactions that are present in this system are:
\begin{align}
H + O_2 &\leftrightarrow OH + O & (&1) \\
O + H_2 &\leftrightarrow OH + H & (&2) \\
OH + H_2 &\leftrightarrow H_2O + H & (&3) \\
H + OH + M &\rightarrow H_2O + M & (&4) \\
H_2 + M &\rightarrow 2H + M & (&5) \\
O_2 + M &\rightarrow 2O + M & (&6)
\end{align}

Then, the rate equations for each species are:

\begin{align}
\frac{d[H]}{dt} &= k_{r,1}[OH][O] + k_{f,2}[O][H_2] + k_{f,3}[OH][H_2] + 2k_{f,5}[H_2][M] - k_{f,1}[H][O2] - k_{r,2}[H][OH] - k_{r,3}[H_2O][H] - k_{f,4}[H][OH][M] \\
\frac{d[O]}{dt} &= k_{f,1}[H][O2] + k_{r,2}[H][OH] + 2k_{f,6}[O_2][M] - k_{r,1}[OH][O] - k_{f,2}[O][H_2] \\
\frac{d[OH]}{dt} &= k_{f,1}[H][O2] + k_{f,2}[O][H_2] + k_{r,3}[H_2O][H] - k_{r,1}[OH][O] - k_{r,2}[H][OH] - k_{f,3}[OH][H_2] - k_{f,4}[H][OH][M] \\
\frac{d[H_2]}{dt} &= k_{r,2}[OH][H] + k_{r,3}[H_2O][H] - k_{f,2}[O][H_2] - k_{f,3}[OH][H_2] - k_{f,5}[H_2][M] \\
\frac{d[O_2]}{dt} &= k_{r,1}[OH][O] - k_{f,1}[H][O2] - k_{f,6}[O_2][M] \\
\frac{d[H_2O]}{dt} &= k_{f,3}[OH][H_2] + k_{f,4}[H][OH][M] - k_{r,3}[H_2O][H]
\end{align}

In [ ]:
%%latex
Under the assumption of partial equilibrium of the three reversible reactions, we know that their
forward and backward rates are the same.
\begin{align}
k_{f,1}[H][O_2] &= k_{r,1}[OH][O] \\
k_{f,2}[O][H_2] &= k_{r,2}[OH][H] \\
k_{f,3}[OH][H_2] &= k_{r,3}[H_2O][H]
\end{align}

Then, the equilibrium rate constants for these, noting that because they're bimolecular, $K_c = K_p$:
\begin{align}
\frac{[OH][O]}{[H][O_2]} &= K_{p,1} \\
\frac{[OH][H]}{[O][H_2]} &= K_{p,2} \\
\frac{[H_2O][H]}{[OH][H_2]} &= K_{p,3}
\end{align}

In [ ]:
import sympy as sym
from IPython.display import display
sym.init_printing(use_latex=True)
H, O, OH, H2, O2, H2O, K1, K2, K3 = sym.symbols('[H] [O] [OH] [H_{2}] [O_{2}] [H_{2}O] K1 K2 K3', positive=True, real=True)
k1f, k1r, k2f, k2r, k3f, k3r, k4f = sym.symbols('k_1f k_1r k_2f k_2r k_3f k_3r k_4f', positive=True, real=True)
K_1 = sym.Eq(OH*O/(H*O2), K1)
K_2 = sym.Eq(OH*H/(O*H2), K2)
K_3 = sym.Eq(H2O*H/(OH*H2), K3)
OH_PE, O_PE, H_PE = sym.solve((K_1, K_2, K_3), (OH, O, H))[0]
display(OH_PE, O_PE, H_PE)

In [ ]:
%%latex
At partial equilibrium, the three reversible reactions cancel and the rate of change of $H_2$, $O_2$ and $H_2O$ are
\begin{align}
\frac{d[H_2]}{dt} &= -k_{f,5}[H_2][M] \\
\frac{d[O_2]}{dt} &= -k_{f,6}[O_2][M] \\
\frac{d[H_2O]}{dt} &= k_{f,4}[H][OH][M] = k_{f,4} K_1 K_2 K_3 \frac{[H_2]^2[O_2]}{[H_2O]}
\end{align}

In [ ]:
dH_2Odt = k4f*H_PE*OH_PE
display(dH_2Odt)

In [ ]:
def ddt(y, t, p):
    """RHS of the differential equations.
    
    Parameters
    ----------
    y: np.array
        Vector of state values (i.e., species concentrations)
    t: float
        Time
    p: list
        Parameters, K1, K2, K3, kf4, kf5, kf6
    """
    M = 8.314*1000.0/10.0E5
    H2, O2, H2O = y
    K1, K2, K3, kf4, kf5, kf6 = p
    f = [
        -kf5*H2*M,
        -kf6*O2*M,
        kf4*K1*K2*K3*H2**2*O2/H2O,
    ]
    return f

In [ ]:
import cantera as ct
gas = ct.Solution('h2o2.xml')
gas.TP = 1000, 10.0E5
Ks = gas.equilibrium_constants
kfs = gas.forward_rate_constants
krs = gas.reverse_rate_constants
eqs = gas.reaction_equations()

In [ ]:
# print(eqs)
indices = [
    0,  # 6
    2,  # 2
    9,  # 1
    10, # 5
    13, # 4
    19, # 3
]
K1 = Ks[indices[2]]
K2 = Ks[indices[1]]
K3 = Ks[indices[5]]
kf4 = kfs[indices[4]]
kf5 = krs[indices[0]]  # Assume H2 rate same as O2 rate
kf6 = krs[indices[0]]

p = [K1, K2, K3, kf4, kf5, kf6]

# Initial conditions
H2_0 = 1.0
O2_0 = 1.0
H2O_0 = 1.0E-3
y_0 = [H2_0, O2_0, H2O_0]
# ODE solver parameters
abserr = 1.0e-8
relerr = 1.0e-6
stoptime = 10.0
numpoints = 250

# Create the time samples for the output of the ODE solver.
# I use a large number of points, only because I want to make
# a plot of the solution that looks nice.
t = [stoptime * float(i) / (numpoints - 1) for i in range(numpoints)]

In [ ]:
from scipy.integrate import odeint
ysol = odeint(ddt, y_0, t, args=(p,), atol=abserr, rtol=relerr)

In [ ]:
print(ysol)

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
plt.plot(t, ysol[:, 0])
plt.plot(t, ysol[:, 1])
plt.plot(t, ysol[:, 2])